In [97]:
import random
import numpy as np
import pandas as pd

# Room draw optimization models using gurobi python API
## 1. Model for multiple preferences with individual parties
Consistent with the output in multi_preferences model in `multi_preferences.mod`

In [110]:
# number parameters
n_rooms = 600
n_people = 600
n_preferences = 5

ROOMS = list(range(1, n_rooms + 1))
PEOPLE = list(range(1, n_people + 1))
PREFERENCES = list(range(1, n_preferences + 1))

# preference and weight parameters
weights = {(p, pr): n_preferences - pr  + 1 for p in PEOPLE for pr in PREFERENCES}



preferences = np.random.randint(low=1,high=n_rooms,size=(n_people, n_preferences))
preferences_lower = preferences
preferences_upper = np.minimum(preferences + np.random.randint(0, 5, (n_rooms, n_preferences)), n_rooms*np.ones((n_rooms, n_preferences))).astype(np.int64)


In [113]:
# save these to text to be interpreted by the models
c = ['PEOPLE', 'PREFERENCE', 'preference_lower', 'preference_upper']
table = np.zeros((n_people*n_preferences, 4), dtype=np.int32)
for i in range(n_people ):
    for j in range(n_preferences):
        table[i*n_preferences + j][0] = i + 1
        table[i*n_preferences + j][1] = j + 1
        table[i*n_preferences + j][2] = preferences_lower[i,j]
        table[i*n_preferences + j][3] = preferences_upper[i,j]
pd.DataFrame(table, columns=c).to_csv('preferences.csv', index=False)

## Simulate Greedy Room Draw
Assume that all agents act independently, selecting the first available room of highest choice

In [114]:
assignment = np.zeros(n_people, dtype=np.int32)
weight = np.zeros(n_people, dtype=np.int32)
for person in range(n_people):
    assigned = False
    for pref in range(n_preferences):
        for i in range(preferences_lower[person, pref]-1, preferences_upper[person, pref]):
            if not assignment[i]:
                assigned = True
                assignment[i] = person
                weight[i] = n_preferences - pref
                break
        if assigned:
            break
    if not assigned:
        for i in range(n_rooms):
            if not assignment[i]:
                assignment[i] = person
                break

In [121]:
greedy_utility = sum(weight)
print(greedy_utility)

2556
